In [ ]:
# Create random subsample from dataset with replacement
def subsample(dataset,ratio):
    sample = list()
    n_sample = round(len(dataset)*ratio)
    while len(sample)<n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [ ]:
# Select the best split point for a dataset
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999,999,999,None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value. 'groups': b_groups}

In [ ]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node,max_depth,min_size,n_features,depth):
    lef,right = node['groups']
    del(node['groups'])
    # check for no split
    if not left or not right:
        node['left'], node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left,n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
        
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'],dict):
            return predict(node['left'],row)
        else:
            return node['left']
    else:
        if isinstance(node['right'],dict):
            return predict(node['right'],row)
        else: 
            return node['right']

In [ ]:
# Make a prediction with a list of bagged trees
def bagging_predict(trees,row):
    predictions = [predict(tree,row) for tree in trees]
    return max(set(predictions), key = predictions.count)

# Random Forest Algorithm
def random_forest(train,test,max_depth,min_size,sample_size,n_trees,n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)